In [54]:
import pandas as pd

# data/Amazon Import - 01.xlsx - import .xlsx file 


In [55]:
# import chardet

# def find_encoding(fname):
#     r_file = open(fname, 'rb').read()
#     result = chardet.detect(r_file)
#     return result['encoding']

# encoding = find_encoding('data/All_Listings_Report_US.txt')
# print(encoding)

In [56]:

import_df_MT = pd.read_excel('data/Amazon Import - 01.xlsx', sheet_name='Master Sku Translation', dtype=str)
# save import_df_MT to csv
import_df_MT.to_csv('data/import_df_MT.csv', index=False)

all_listings_US  = pd.read_csv('data/All_Listings_Report_US.txt', sep='\t', encoding='UTF-8-SIG', dtype=str)
all_listings_CA = pd.read_csv('data/All_Listings_Report_CA.txt', sep='\t', encoding='UTF-8-SIG', dtype=str)
all_listings_MX = pd.read_csv('data/All_Listings_Report_MX.txt', sep='\t', encoding='UTF-8-SIG', dtype=str)

BS_export_df = pd.read_csv('data/STOCK-STATUS202407098.952568.TXT', sep='\t', encoding='ascii', skiprows=2, dtype=str)

#save BS_export_df to csv
BS_export_df.to_csv('data/BS_export_df.csv', index=False)

print("import_df_MT columns: " , import_df_MT.columns,"\n")
# print("all_listings_US columns: " , all_listings_US.columns,"\n")
# print("all_listings_CA columns: " , all_listings_CA.columns,"\n")
# print("all_listings_MX columns: " , all_listings_MX.columns,"\n")
# print("BS_export_df columns: " , BS_export_df.columns,"\n")

all_listings_MX.to_csv('data/All_Listings_Report_MX.csv', index=False)


# BS_sku_list = BS_export_df['CODE'].str.strip().tolist()


import_df_MT columns:  Index(['FIXED FOR LOOKUP', 'seller-sku', 'Sku No Underscore', 'SKU (FINAL)',
       'Type', 'Product', 'Product NZ', 'Marketplace Source', 'File',
       'Done By', 'Sku (FINAL) FIXED'],
      dtype='object') 



In [57]:

def combine_all_listings_by_SKU_NA(us_df, ca_df, mx_df):
    """
    This function combines product listings from the US, Canada, and Mexico into a single DataFrame.
    Each country's DataFrame is processed to retain only the 'seller-sku', 'asin1', 'status', and 'fulfillment-channel' columns.
    The columns are then renamed to indicate the country of origin.
    The processed DataFrames are then merged on the 'seller_sku' column.

    Parameters:
    us_df (pandas.DataFrame): DataFrame containing product listings from the US.
    ca_df (pandas.DataFrame): DataFrame containing product listings from Canada.
    mx_df (pandas.DataFrame): DataFrame containing product listings from Mexico.

    Returns:
    result_df (pandas.DataFrame): A DataFrame containing the combined product listings. Each row represents a unique product listing.
    The DataFrame contains the following columns: 'seller_sku', 'ASIN_US', 'status_US', 'fulfillment_US', 'ASIN_CA', 'status_CA', 'fulfillment_CA', 'ASIN_MX', 'status_MX', 'fulfillment_MX'.
    """

    def process_country(df, country):
        df = df[['seller-sku', 'asin1', 'status', 'fulfillment-channel']].copy()
        df.columns = ['seller_sku', f'ASIN_{country}', f'status_{country}', f'fulfillment_{country}']
        return df

    dfs = [
        process_country(us_df, 'US'),
        process_country(ca_df, 'CA'),
        process_country(mx_df, 'MX')
    ]

    result_df = dfs[0]
    for df in dfs[1:]:
        result_df = pd.merge(result_df, df, on='seller_sku', how='outer')

    return result_df


combined_df = combine_all_listings_by_SKU_NA(all_listings_US, all_listings_CA, all_listings_MX)

# # Вывод первых нескольких строк результата
# print(combined_df.head())

# Сохранение результата в CSV файл в папке results
combined_df.to_csv('results/combined_listings.csv', index=False)



In [58]:
# clean up the data from import_df_MT 
amz_sku_mapping = import_df_MT[['seller-sku', 'SKU (FINAL)', 'Product','Product NZ']].copy()
amz_sku_mapping = amz_sku_mapping.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# rename columns to ['seller-sku', 'BS_SKU', 'B_SKU','B_NZ_SKU']
amz_sku_mapping.columns = ['seller_sku', 'BS_SKU', 'B_SKU','B_NZ_SKU']

# merge to combined_df
final_NA_mapping = pd.merge(combined_df, amz_sku_mapping, on='seller_sku', how='left')

#save final_NA_mapping to csv file

final_NA_mapping.to_csv('results/final_NA_mapping.csv', index=False)


In [68]:
import gspread
from google.oauth2.service_account import Credentials

def df_to_sheet_list(df):
    df = df.fillna('')
    df = [df.columns.values.tolist()] + df.values.tolist()
    return df

scopes = ["https://www.googleapis.com/auth/spreadsheets"]
creds = Credentials.from_service_account_file("../credentials/sheets_api_cred.json", scopes=scopes)
client = gspread.authorize(creds)

sheet_id = "1ZMzIMn7CzV_tUJSfXguHYLh3fkkgHVh_0u2NBWCzEAQ"
workbook = client.open_by_key(sheet_id)

worksheet_list = map(lambda x: x.title, workbook.worksheets())
new_worksheet_name = "final_NA_mapping"

if new_worksheet_name in worksheet_list:
    sheet = workbook.worksheet(new_worksheet_name)
else:
    sheet = workbook.add_worksheet(new_worksheet_name, rows=10, cols=10)
sheet.clear()

def update_sheet_in_chunks(sheet, df, chunk_size=1000):
    df_list = df_to_sheet_list(df)
    for i in range(0, len(df_list), chunk_size):
        chunk = df_list[i:i+chunk_size]
        range_name = f'A{i+1}:Z{i+len(chunk)}'
        sheet.update(chunk, range_name)

update_sheet_in_chunks(sheet, final_NA_mapping)

In [66]:


print(df_to_sheet_list(final_NA_mapping)[:5])

[['seller_sku', 'ASIN_US', 'status_US', 'fulfillment_US', 'ASIN_CA', 'status_CA', 'fulfillment_CA', 'ASIN_MX', 'status_MX', 'fulfillment_MX', 'BS_SKU', 'B_SKU', 'B_NZ_SKU'], ['00-3UDR-PUQ3', 'B0088WNXBK', 'Active', 'DEFAULT', 'B0088WNXBK', 'Active', 'DEFAULT', 'B0088WNXBK', 'Active', 'DEFAULT', '', '', ''], ['00-ACLU-JCYV', 'B07S9FY2ML', 'Inactive', 'AMAZON_NA', 'B07S9FY2ML', 'Inactive', 'AMAZON_NA', 'B07S9FY2ML', 'Inactive', 'AMAZON_NA', '', '3025 BB A 6"', '3025 BB A 6"'], ['00-CD5D-G3U5', 'B008I6X302', 'Active', 'DEFAULT', 'B008I6X302', 'Active', 'DEFAULT', 'B008I6X302', 'Active', 'DEFAULT', '', '73987 HT', '73987 HT'], ['00-YASY-W9K8', 'B0088WNXA6', 'Active', 'DEFAULT', 'B0088WNXA6', 'Active', 'DEFAULT', 'B0088WNXA6', 'Active', 'DEFAULT', '', '3393 ZB BD', '3393 ZB BD']]
